# Prompt Creation from Filtered Metadata

This notebook:
1. Loads `metadata_filtered_by_maps.csv`.
2. Removes a specific set of tags (`relatable`, `thoughtprovoking`, `minimalist`, etc.).
3. Generates a new `prompt` column using the format: `"minimalistic logo, category: {category_main}, description: {description}, tags: {tags}"`.
4. Replaces underscores with spaces in the final prompt.
5. Saves the result to `final_prompts.csv`.
6. Displays a few random, non-truncated prompts for quality checking.

In [1]:
# Imports & Paths
from pathlib import Path
import pandas as pd
import os

# Set display options to show full prompts
pd.set_option('display.max_colwidth', None)

BASE_PATH = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
PROJECT_ROOT = BASE_PATH / '../..'
INPUT_DATA_DIR = PROJECT_ROOT / 'output' / 'amazing_logos_v4' / 'data' / 'meta_postprep'
OUTPUT_PATH = INPUT_DATA_DIR / 'final_prompts.csv'
INPUT_PATH = INPUT_DATA_DIR / 'metadata_filtered_by_maps.csv'

print('Input path:', INPUT_PATH)
print('Output path:', OUTPUT_PATH)

if not INPUT_PATH.exists():
    raise FileNotFoundError(f"Input file not found. Please run the filtering notebook first: {INPUT_PATH}")

df = pd.read_csv(INPUT_PATH)
print('Loaded filtered metadata rows:', len(df))
df.head()

Input path: c:\studium\master_thesis\data_prep\notebooks\meta_postprep\..\..\output\amazing_logos_v4\data\meta_postprep\metadata_filtered_by_maps.csv
Output path: c:\studium\master_thesis\data_prep\notebooks\meta_postprep\..\..\output\amazing_logos_v4\data\meta_postprep\final_prompts.csv
Loaded filtered metadata rows: 1810


,id,category_main,category,description,tags,company
0,amazing_logo_v4000185,manufacturing_transport,textiles_manufacturing,Drop Oval United states Water,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Santa Fe By Design
1,amazing_logo_v4000313,food_beverage,restaurant_dining,Bird Fork United states,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Bluebird Cafe
2,amazing_logo_v4000469,retail_hospitality,home_improvement,A I Italy N,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Collezione Nai
3,amazing_logo_v4000481,manufacturing_transport,automotive_transport,A Arrow France Line Triangle,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Renault Alpine
4,amazing_logo_v4000523,entertainment_sports_media,arts_culture,Circle Germany Line Rotation Spiral Swirl,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Rotation


In [2]:
# Define tags to remove and clean the tags column
TAGS_TO_REMOVE = {
    'relatable',
    'thoughtprovoking',
    'minimalist',
    'recognizable',
    'successful_vibe',
}

def clean_tags(tags_str):
    if pd.isna(tags_str):
        return ''
    
    # Split, strip, and filter
    tags_list = [tag.strip() for tag in tags_str.split(',')]
    filtered_tags = [tag for tag in tags_list if tag not in TAGS_TO_REMOVE and tag]
    
    return ', '.join(filtered_tags)

# Apply the cleaning function
df['tags_cleaned'] = df['tags'].apply(clean_tags)

print("Original vs. Cleaned Tags:")
df[['tags', 'tags_cleaned']].head()

Original vs. Cleaned Tags:


,tags,tags_cleaned
0,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white","abstract, sharp, vector_art, even_edges, black_and_white"
1,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white","abstract, sharp, vector_art, even_edges, black_and_white"
2,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white","abstract, sharp, vector_art, even_edges, black_and_white"
3,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white","abstract, sharp, vector_art, even_edges, black_and_white"
4,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white","abstract, sharp, vector_art, even_edges, black_and_white"


In [3]:
# Generate the prompt
def create_prompt(row):
    # Start with the base phrase
    parts = ["minimalistic logo, solid background"]
    
    # Add category
    # category = str(row['category_main']).replace('_', ' ') if pd.notna(row['category_main']) else ''
    # if category:
    #     parts.append(f"category: {category}")
        
    # Add description
    description = str(row['description']).replace('_', ' ') if pd.notna(row['description']) else ''
    if description:
        parts.append(f"description: {description}")
        
    # Add cleaned tags
    tags = str(row['tags_cleaned']).replace('_', ' ') if pd.notna(row['tags_cleaned']) else ''
    if tags:
        parts.append(f"tags: {tags}")
        
    return "; ".join(parts)

df['prompt'] = df.apply(create_prompt, axis=1)

print("Generated Prompts:")
df[['id', 'prompt']].head()

Generated Prompts:


,id,prompt
0,amazing_logo_v4000185,"minimalistic logo, solid background; description: Drop Oval United states Water; tags: abstract, sharp, vector art, even edges, black and white"
1,amazing_logo_v4000313,"minimalistic logo, solid background; description: Bird Fork United states; tags: abstract, sharp, vector art, even edges, black and white"
2,amazing_logo_v4000469,"minimalistic logo, solid background; description: A I Italy N; tags: abstract, sharp, vector art, even edges, black and white"
3,amazing_logo_v4000481,"minimalistic logo, solid background; description: A Arrow France Line Triangle; tags: abstract, sharp, vector art, even edges, black and white"
4,amazing_logo_v4000523,"minimalistic logo, solid background; description: Circle Germany Line Rotation Spiral Swirl; tags: abstract, sharp, vector art, even edges, black and white"


In [4]:
# Reorder columns and save the final dataframe
# Keep original columns but place 'prompt' after 'id'
original_cols = df.columns.tolist()
# Remove the new intermediate and prompt columns to re-insert them
original_cols.remove('tags_cleaned')
original_cols.remove('prompt')

final_cols = ['id', 'prompt'] + [col for col in original_cols if col != 'id']

final_df = df[final_cols]

# Save to CSV
final_df.to_csv(OUTPUT_PATH, index=False)

print(f"Saved final data with prompts to: {OUTPUT_PATH}")
print(f"Total rows: {len(final_df)}")
final_df.head()

Saved final data with prompts to: c:\studium\master_thesis\data_prep\notebooks\meta_postprep\..\..\output\amazing_logos_v4\data\meta_postprep\final_prompts.csv
Total rows: 1810


,id,prompt,category_main,category,description,tags,company
0,amazing_logo_v4000185,"minimalistic logo, solid background; description: Drop Oval United states Water; tags: abstract, sharp, vector art, even edges, black and white",manufacturing_transport,textiles_manufacturing,Drop Oval United states Water,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Santa Fe By Design
1,amazing_logo_v4000313,"minimalistic logo, solid background; description: Bird Fork United states; tags: abstract, sharp, vector art, even edges, black and white",food_beverage,restaurant_dining,Bird Fork United states,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Bluebird Cafe
2,amazing_logo_v4000469,"minimalistic logo, solid background; description: A I Italy N; tags: abstract, sharp, vector art, even edges, black and white",retail_hospitality,home_improvement,A I Italy N,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Collezione Nai
3,amazing_logo_v4000481,"minimalistic logo, solid background; description: A Arrow France Line Triangle; tags: abstract, sharp, vector art, even edges, black and white",manufacturing_transport,automotive_transport,A Arrow France Line Triangle,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Renault Alpine
4,amazing_logo_v4000523,"minimalistic logo, solid background; description: Circle Germany Line Rotation Spiral Swirl; tags: abstract, sharp, vector art, even edges, black and white",entertainment_sports_media,arts_culture,Circle Germany Line Rotation Spiral Swirl,"successful_vibe,minimalist,thoughtprovoking,abstract,recognizable,relatable,sharp,vector_art,even_edges,black_and_white",Simple elegant logo for Rotation


In [5]:
# Showcase a few random prompts (not truncated)
print("--- Random Sample Prompts ---")
sample_prompts = final_df['prompt'].sample(5, random_state=42)
for prompt in sample_prompts:
    print(f"- {prompt}\n")

--- Random Sample Prompts ---
- minimalistic logo, solid background; description: Care Circle Holistic Leaf Healthcare Hands; tags: abstract, sharp, vector art, even edges

- minimalistic logo, solid background; description: fitness leap jaguar; tags: abstract, sharp, vector art, even edges

- minimalistic logo, solid background; description: advertising graphic design branding creative design marketing business; tags: abstract, sharp, vector art, even edges

- minimalistic logo, solid background; description: Blue Currier Graph Sans Modern Green Chart Lines Serif Bar Simple Smith Ryan Stacked Micro Analytics Clean; tags: abstract, sharp, vector art, even edges

- minimalistic logo, solid background; description: gray handwritten wordmark; tags: abstract, sharp, vector art, even edges

